# Airline plannning problem

## Variable

$C$ = Set of cities in this example it is A,B,C,D

$P$ = Set of plate type B707, (Electra's) & (DC9's).

$s_c$ = no of times need to serve city c

$a_{p}$ = avg time flight time limit for each plane i.e 18 hrs a day

$n_p$ = no of flights available 

$time_{c,p}$ = avg flight travel time between hub to city 

$revenue_{c,p}$ = revenue from each trip

$cost_{c,p}$ = trip cost for each trip 


$x_{c,p}$ = No of fligt type (p) trip from hub to location (c) 


## Constraints
\begin{equation}
\sum_{p \in P} x_{c,p} = s_{c} \hspace{2cm} \forall c \in C \tag{serve constraints}
\end{equation}  


\begin{equation}
\sum_{p \in P} x_{c,p} * time_{c,p} \leq a_{p} * n_{p} \hspace{2cm} \forall c \in C \tag{time constraints}
\end{equation}


## Objective Function
### Cost Minimization 

\begin{equation}
\sum_{c \in C} \sum_{p \in P} x_{c,p}* cost_{c,p} 
\end{equation}

### profit maximization 
\begin{equation}
\sum_{c \in C} \sum_{p \in P} x_{c,p}* (revenue_{c,p}-cost_{c,p}) 
\end{equation}

### time minimization 
\begin{equation}
\sum_{c \in C} \sum_{p \in P} x_{c,p}* (time_{c,p}) 
\end{equation}

In [62]:
import pandas as pd
data = pd.read_csv('data5.csv')

In [91]:
from pulp import LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpMaximize

In [98]:
# Define the set index
city  = ['A', 'B', 'C', 'D']
plane = ['B707', 'DC', 'Electra']


n = {'B707':10, 'DC':2, 'Electra':15}

cp_index = []
for c in city:
    for p in plane:
        cp_index.append((c,p))

cost, revenue, time = {} , {}, {}       
for i,j in zip(data['Plane'],data['City']):
    cost[(j,i)] = data[(data['City']==j)&(data['Plane']==i)]['Cost'].iloc[0]
    revenue[(j,i)] = data[(data['City']==j)&(data['Plane']==i)]['Revenue'].iloc[0]
    time[(j,i)] = data[(data['City']==j)&(data['Plane']==i)]['Avg time'].iloc[0]

# serve 
s = {'A':4, 'B':4, 'C':4, 'D':2}



In [99]:
x_var['A']['B707']

x_A_B707

In [100]:
cost

{('A', 'B707'): 6000,
 ('B', 'B707'): 7000,
 ('C', 'B707'): 8000,
 ('D', 'B707'): 10000,
 ('A', 'Electra'): 1000,
 ('B', 'Electra'): 2000,
 ('C', 'Electra'): 4000,
 ('D', 'Electra'): 0,
 ('A', 'DC'): 2000,
 ('B', 'DC'): 3500,
 ('C', 'DC'): 6000,
 ('D', 'DC'): 10000}

### Cost Minimization

In [184]:
# Define model variable
x_var = LpVariable.dicts("x", (city, plane), lowBound=0, cat="Continuous")

# Define objective function
model = LpProblem(name="Airline-Planning", sense=LpMinimize)
obj_func = lpSum([x_var[c][p]*cost[(c,p)] for c in city for p in plane])
model += obj_func

# Define constraint
for c in city:
    model += lpSum([x_var[c][p] for p in plane]) == s[c]
    
for p in plane:
    model += lpSum([x_var[c][p]*time[(c,p)] for c in city]) <= 18*n[p]

model += x_var['D']['Electra'] ==0


In [185]:
model

Airline-Planning:
MINIMIZE
6000*x_A_B707 + 2000*x_A_DC + 1000*x_A_Electra + 7000*x_B_B707 + 3500*x_B_DC + 2000*x_B_Electra + 8000*x_C_B707 + 6000*x_C_DC + 4000*x_C_Electra + 10000*x_D_B707 + 10000*x_D_DC + 0
SUBJECT TO
_C1: x_A_B707 + x_A_DC + x_A_Electra = 4

_C2: x_B_B707 + x_B_DC + x_B_Electra = 4

_C3: x_C_B707 + x_C_DC + x_C_Electra = 4

_C4: x_D_B707 + x_D_DC + x_D_Electra = 2

_C5: x_A_B707 + 2 x_B_B707 + 5 x_C_B707 + 10 x_D_B707 <= 180

_C6: x_A_DC + 2 x_B_DC + 6 x_C_DC + 12 x_D_DC <= 36

_C7: 2 x_A_Electra + 4 x_B_Electra + 8 x_C_Electra + 20 x_D_Electra <= 270

_C8: x_D_Electra = 0

VARIABLES
x_A_B707 Continuous
x_A_DC Continuous
x_A_Electra Continuous
x_B_B707 Continuous
x_B_DC Continuous
x_B_Electra Continuous
x_C_B707 Continuous
x_C_DC Continuous
x_C_Electra Continuous
x_D_B707 Continuous
x_D_DC Continuous
x_D_Electra Continuous

In [186]:
status = model.solve()
status

1

In [187]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [188]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

x_A_B707: 0.0
x_A_DC: 0.0
x_A_Electra: 4.0
x_B_B707: 0.0
x_B_DC: 0.0
x_B_Electra: 4.0
x_C_B707: 0.0
x_C_DC: 0.0
x_C_Electra: 4.0
x_D_B707: 0.0
x_D_DC: 2.0
x_D_Electra: 0.0


In [189]:
obj_func.value()

48000.0

In [190]:
model.solve()

1

In [191]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

x_A_B707: 0.0
x_A_DC: 0.0
x_A_Electra: 4.0
x_B_B707: 0.0
x_B_DC: 0.0
x_B_Electra: 4.0
x_C_B707: 0.0
x_C_DC: 0.0
x_C_Electra: 4.0
x_D_B707: 0.0
x_D_DC: 2.0
x_D_Electra: 0.0


In [192]:
obj_func.value()

48000.0

### Profit Maximization

In [193]:
x_var = LpVariable.dicts("x", (city, plane), lowBound=0, cat="Continuous")

model = LpProblem(name="Airline-Planning", sense=LpMaximize)

obj_func2 = lpSum([x_var[c][p]*(revenue[(c,p)]-cost[(c,p)]) for c in city for p in plane])
model += obj_func2
    
# Define constraint
for c in city:
    model += lpSum([x_var[c][p] for p in plane]) >= s[c]
    
for p in plane:
    model += lpSum([x_var[c][p]*time[(c,p)] for c in city]) <= 18*n[p]

model += x_var['D']['Electra'] ==0


In [194]:
model.solve()

1

In [195]:
print(f"status: {model.status}, {LpStatus[model.status]}")

status: 1, Optimal


In [196]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

x_A_B707: 0.0
x_A_DC: 36.0
x_A_Electra: 135.0
x_B_B707: 4.0
x_B_DC: 0.0
x_B_Electra: 0.0
x_C_B707: 4.0
x_C_DC: 0.0
x_C_Electra: 0.0
x_D_B707: 15.2
x_D_DC: 0.0
x_D_Electra: 0.0


In [197]:
obj_func2.value()

471600.0

as you can see x_D_B707: 15.2 which can't be possible so we need to solve Integer Programming for this by restricting x_var to integer

In [198]:
x_var = LpVariable.dicts("x", (city, plane), lowBound=0, cat="Integer")
model = LpProblem(name="Airline-Planning", sense=LpMaximize)

obj_func2 = lpSum([x_var[c][p]*(revenue[(c,p)]-cost[(c,p)]) for c in city for p in plane])
model += obj_func2
    
# Define constraint
for c in city:
    model += lpSum([x_var[c][p] for p in plane]) >= s[c]
    
for p in plane:
    model += lpSum([x_var[c][p]*time[(c,p)] for c in city]) <= 18*n[p]

model += x_var['D']['Electra'] ==0


In [199]:
model.solve()

1

In [200]:
obj_func2.value()

470000.0

In [201]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

x_A_B707: 0.0
x_A_DC: 36.0
x_A_Electra: 135.0
x_B_B707: 5.0
x_B_DC: 0.0
x_B_Electra: 0.0
x_C_B707: 4.0
x_C_DC: 0.0
x_C_Electra: 0.0
x_D_B707: 15.0
x_D_DC: 0.0
x_D_Electra: 0.0


### Time Minimization

In [205]:
x_var = LpVariable.dicts("x", (city, plane), lowBound=0, cat="Integer")


model = LpProblem(name="Airline-Planning-3", sense=LpMaximize)


obj_func3 = lpSum([x_var[c][p]*(time[(c,p)]) for c in city for p in plane])
model += obj_func3

# Define constraint
for c in city:
    model += lpSum([x_var[c][p] for p in plane]) == s[c]
    
for p in plane:
    model += lpSum([x_var[c][p]*time[(c,p)] for c in city]) <= 18*n[p]

model += x_var['D']['Electra'] ==0

In [206]:
model.solve()
print(f"status: {model.status}, {LpStatus[model.status]}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

status: 1, Optimal
x_A_B707: 0.0
x_A_DC: 0.0
x_A_Electra: 4.0
x_B_B707: 0.0
x_B_DC: 0.0
x_B_Electra: 4.0
x_C_B707: 0.0
x_C_DC: 0.0
x_C_Electra: 4.0
x_D_B707: 0.0
x_D_DC: 2.0
x_D_Electra: 0.0


In [207]:
obj_func.value()

48000.0